In [ ]:
library(tidyverse)
library(ggplot2)
library(tseries)
library(forecast)
library(lubridate)
library(prophet)
library(corrplot)
library(caret)

In [ ]:
## reading data
train <- read.csv("../input/demand-forecasting-kernels-only/train.csv" , header = TRUE , sep = ",")
test <- read.csv("../input/demand-forecasting-kernels-only/test.csv" , header = TRUE , sep = ",")
all_data <- train %>% bind_rows(test)

sample_submission = read.csv("../input/demand-forecasting-kernels-only/sample_submission.csv" , header = TRUE , sep = ",")

In [ ]:
library(lubridate)

all_data <- train %>% bind_rows(test)

all_data$month <- month(ymd(all_data$date))
all_data$weekday <- wday(ymd(all_data$date))
all_data$quarter <- quarter(ymd(all_data$date))
all_data$quarter <- quarter(ymd(all_data$date))
all_data$day <- day(ymd(all_data$date))

head(all_data)


In [ ]:
test <- all_data[is.na(all_data$sales),-c(4,5)]
test_labels <- all_data[is.na(all_data$sales),5]
train <- all_data[!is.na(all_data$sales),-(5)]

In [ ]:
intrain<-createDataPartition(y=train$sales,p=0.7,list=FALSE)
training<-train[intrain,]
testing<-train[-intrain,]

In [ ]:
summary(train$sales)
hist(train$sales , breaks = 20)

train$date1 = as.Date(train$date)
test$date1 = as.Date(test$date)

train$date = NULL
test$date = NULL


stationary_test = adf.test(diff(train$sales), alternative="stationary", k=0)
print(stationary_test$p.value)

sales_ma = ts(train$sales , frequency = 30)
acf(sales_ma , plot = TRUE)
pacf(sales_ma, plot = TRUE)

model_0 = arima(train$sales , order = c(1,1,5))

tsdisplay(residuals(model_0), lag.max=15, main='Seasonal Model Residuals')

prediction = predict(model_0 , n.ahead = 45000)

In [ ]:
sample_submission$sales = prediction$pred
write_csv(submission, "./mysubmission1.csv")

In [ ]:
library(caret)
library(dplyr)
library(DataExplorer)
library(MLmetrics)
library(prophet)

m <- prophet(daily.seasonality=TRUE, n_changepoints = 0)

m <- add_regressor(m, 'store')

m <- add_regressor(m, 'item')

m <- fit.prophet(m, train_2)

forecast <- predict(m, test_2)

pred_frame <- data.frame(id = test$id, sales = forecast$yhat)

head(pred_frame)

write_csv(pred_frame, "mysubmission2.csv")
